<a href="https://colab.research.google.com/github/fluanceifi/NoTeamNurd/blob/develop/face_parsing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2


In [ ]:
pip install git+https://github.com/FacePerceiver/facer.git@main

In [ ]:
from google.colab import files

# 파일 업로드
uploaded = files.upload()


In [ ]:
import torch
import facer
import numpy as np
import matplotlib.pyplot as plt

# 1. 이미지 로드 및 face parsing
device = 'cuda' if torch.cuda.is_available() else 'cpu'
image = facer.hwc2bchw(facer.read_hwc('/content/woojin.png')).to(device)

face_detector = facer.face_detector('retinaface/mobilenet', device=device)
with torch.inference_mode():
    faces = face_detector(image)

face_parser = facer.face_parser('farl/lapa/448', device=device)
with torch.inference_mode():
    faces = face_parser(image, faces)

In [ ]:
# 2. 세그멘테이션 결과
seg_logits = faces['seg']['logits']
seg_probs = seg_logits.softmax(dim=1)
parsed_classes = seg_probs.argmax(dim=1)

In [ ]:
# 3. 추출하고 싶은 부위 리스트 (좌우 통합 포함)
merged_labels = {
    'face_skin': [1],
    'eyebrows': [2, 3],
    'eyes': [4, 5],
    'nose': [6],
    'lips': [7, 9],
    'inner_mouth': [8],
    'hair': [10]
}

In [ ]:
# 4. 부위별 추출
extracted_parts = {}

for part_name, class_indices in merged_labels.items():
    mask = torch.zeros_like(parsed_classes, dtype=torch.bool)
    for idx in class_indices:
        mask |= (parsed_classes == idx)
    part_image = image * mask.unsqueeze(1).float()  # [B, C, H, W]
    extracted_parts[part_name] = part_image

In [ ]:
plt.figure(figsize=(16, 10))
for i, (part_name, part_img) in enumerate(extracted_parts.items()):
    plt.subplot(2, 4, i + 1)
    facer.show_bchw(part_img)
    plt.title(part_name)
plt.tight_layout()
plt.show()
## 여기까지 부위 추출 관련 코드 - 그 다음부터는 색 추출

In [ ]:
print(image.shape)

In [ ]:
# 원본이미지 [1, 3, H, W] → NumPy [H, W, 3] - 원본이미지를 NumPy에서 쓰기 위해서 변환하는과정.
image_np = image[0].permute(1, 2, 0).cpu().numpy()  # shape: (506, 524, 3)

plt.imshow(image_np)

In [ ]:
import numpy as np
import cv2

# 1. 피부 부위 마스크 생성 (클래스 번호 1 = face_skin) - True / False 로 구분된 이미지 - 피부부위 마스크를 만들기 위해서
skin_mask_tensor = (parsed_classes == 1)
skin_mask = skin_mask_tensor.squeeze().cpu().numpy().astype(bool)  # shape: (H, W)
plt.imshow(skin_mask)

In [ ]:
skin_pixels = image_np[skin_mask]


# 마스크에서 피부라고 인식한 이미지에 해당하는 픽셀의 값의 평균을 구한다.
if skin_pixels.shape[0] == 0:
    print("❗️마스크에 해당하는 픽셀이 없습니다.")
else:
    mean_rgb = np.mean(skin_pixels, axis=0)
    print(f"📊 평균 RGB 값: R={mean_rgb[0]:.1f}, G={mean_rgb[1]:.1f}, B={mean_rgb[2]:.1f}")

